### 4. **Настройка OAuth 2.0 в Django (без сторонних инструментов)**

#### Шаг 1. **Добавление middleware и маршрутов**

Убедитесь, что в вашем проекте уже подключены все необходимые приложения и middleware, как мы делали ранее:

1. В `INSTALLED_APPS` (`settings.py`):
   ```python
   INSTALLED_APPS = [
       ...
       'oauth2_provider',  # Подключаем поддержку OAuth 2.0
       'rest_framework',  # Для создания API
   ]
   ```

2. В `urls.py`:
   ```python
   from django.urls import path, include

   urlpatterns = [
       ...
       path('o/', include('oauth2_provider.urls', namespace='oauth2_provider')),  # OAuth 2.0 маршруты
   ]
   ```


#### Шаг 2. **Настройка отображения токенов в админке**

Для того чтобы токены были видны в админке, их нужно зарегистрировать.

1. **Откройте `admin.py` вашего приложения**:
   Если файла `admin.py` ещё нет, создайте его.

2. **Добавьте отображение моделей токенов**:
   ```python
   from django.contrib import admin
   from oauth2_provider.models import AccessToken, Application, RefreshToken

   # Регистрируем модели в админке
   admin.site.register(AccessToken)
   admin.site.register(RefreshToken)
   admin.site.register(Application)
   ```

Теперь в админке вы сможете видеть:
- **Applications** (приложения, которые запрашивают токены).
- **Access Tokens** (токены доступа).
- **Refresh Tokens** (обновляющие токены).


#### Шаг 3. **Создание защищённого API**

1. **Создайте защищённый API**:
   Откройте `views.py` вашего приложения и создайте класс:
   ```python
   from rest_framework.views import APIView
   from rest_framework.response import Response
   from rest_framework.permissions import IsAuthenticated
   from oauth2_provider.contrib.rest_framework import OAuth2Authentication

   class ProtectedView(APIView):
       authentication_classes = [OAuth2Authentication]  # Используем OAuth2 для аутентификации
       permission_classes = [IsAuthenticated]  # Доступ только авторизованным пользователям

       def get(self, request):
           return Response({"message": "Вы успешно авторизовались!", "user": request.user.username})
   ```

2. **Добавьте маршрут**:
   В `urls.py` приложения подключите этот эндпоинт:
   ```python
   from django.urls import path
   from .views import ProtectedView

   urlpatterns = [
       ...
       path('api/protected/', ProtectedView.as_view(), name='protected'),
   ]
   ```


#### Шаг 4. **Создание приложения и токена через админку**

Теперь все действия выполняются только через админку:

1. **Создайте приложение (Application)**:
   - Перейдите в админку: `http://127.0.0.1:8000/admin/`.
   - Откройте раздел **Applications** и добавьте новое приложение:
     - **Client type**: `Confidential`.
     - **Authorization grant type**: `Resource owner password-based`.
     - **Name**: укажите имя, например, `My Test App`.
   - Сохраните приложение.

2. **Создайте токен вручную**:
   - В админке перейдите в раздел **Access Tokens**.
   - Нажмите **Add** и создайте новый токен:
     - **User**: выберите пользователя (например, `admin`).
     - **Application**: выберите созданное приложение.
     - **Token**: укажите любое значение (например, `testtoken123`).
     - **Expires**: установите дату, до которой токен будет действителен.

Теперь токен доступен в админке, и вы можете использовать его!

#### Шаг 5. **Проверка защищённого API**

1. **Откройте браузер**:
   Перейдите по следующему адресу, заменив `your_token` на значение токена, который вы создали в админке:
   ```
   http://127.0.0.1:8000/api/protected/?access_token=your_token
   ```

2. **Ожидаемый результат**:
   Если токен правильный, вы увидите сообщение:
   ```json
   {
       "message": "Вы успешно авторизовались!",
       "user": "admin"
   }
   ```

Если токен недействителен или отсутствует, вы получите ошибку:
```json
{
    "error": "Invalid token."
}
```


**Проверка работы**

- Теперь вы можете перейти в браузере по адресу (заменив ACCESS_TOKEN на токен, который вы получили):

```
http://127.0.0.1:8000/api/protected/?access_token=ACCESS_TOKEN
```

Если всё настроено правильно, вы увидите JSON-ответ